<a href="https://colab.research.google.com/github/hchaparov/Dynamic_Pricing_MARL/blob/main/2A_Tennis_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install pettingzoo[atari]
!pip install stable-baselines3>=2.0.0
!pip install supersuit>=3.9.0
!pip install autorom
!apt-get install -y ffmpeg
!pip install moviepy
!apt-get update


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [10]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms
	/usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.


In [11]:
from google.colab import drive
drive.mount('/content/drive')

from pettingzoo.atari import tennis_v3
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
import supersuit as ss
from IPython.display import HTML
from base64 import b64encode
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
from stable_baselines3.common.monitor import Monitor


# Create the parallel environment with render_mode set to 'rgb_array'
env = tennis_v3.parallel_env(render_mode='rgb_array')
env = ss.color_reduction_v0(env, mode='B')
env = ss.resize_v1(env, x_size=84, y_size=84)
env = ss.frame_stack_v1(env, 3)

# Convert ParallelEnv to VecEnv
vec_env = ss.pettingzoo_env_to_vec_env_v1(env)
vec_env = ss.concat_vec_envs_v1(vec_env, 1, num_cpus=1, base_class='stable_baselines3')


# Define paths for saving models and evaluation results
left_model_path = "/content/drive/MyDrive/ppo_tennis_left_test"
right_model_path = "/content/drive/MyDrive/ppo_tennis_right_test"
evaluation_results_path = "/content/drive/MyDrive/evaluation_results_test.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Load or create models for both agents
if os.path.exists(left_model_path + ".zip") and os.path.exists(right_model_path + ".zip"):
  model_left = PPO.load(left_model_path, env= vec_env)
  model_right = PPO.load(right_model_path, env= vec_env)
else:
  model_left = PPO("CnnPolicy", vec_env, verbose=1)
  model_right = PPO("CnnPolicy", vec_env, verbose=1)


Using cpu device
Wrapping the env in a VecTransposeImage.
Using cpu device
Wrapping the env in a VecTransposeImage.


In [ ]:
"""
# Function to save evaluation results to a CSV file
def save_evaluation_results(filename, iteration, avg_reward_left, avg_reward_right):
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = ['iteration', 'avg_reward_left', 'avg_reward_right']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()  # file doesn't exist yet, write a header
        writer.writerow({'iteration': iteration, 'avg_reward_left': avg_reward_left, 'avg_reward_right': avg_reward_right})

# Function to load evaluation results from a CSV file
def load_evaluation_results(filename):
    iterations = []
    avg_rewards_left = []
    avg_rewards_right = []
    if os.path.isfile(filename):
        with open(filename, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                iterations.append(int(row['iteration']))
                avg_rewards_left.append(float(row['avg_reward_left']))
                avg_rewards_right.append(float(row['avg_reward_right']))
    return iterations, avg_rewards_left, avg_rewards_right

"""

In [ ]:
"""
# Function to evaluate the performance of the agents
def evaluate_self_play(model_left, model_right, env, n_eval_episodes):
    total_rewards_left = []
    total_rewards_right = []

    for episode in range(n_eval_episodes):
        obs = env.reset()
        done = [False, False]
        episode_rewards_left = 0
        episode_rewards_right = 0

        while not all(done):
            action_left, _ = model_left.predict(obs[0], deterministic=True)
            action_right, _ = model_right.predict(obs[1], deterministic=True)
            obs, rewards, dones, _ = env.step([action_left, action_right])
            episode_rewards_left += rewards[0]
            episode_rewards_right += rewards[1]
            done = dones

        total_rewards_left.append(episode_rewards_left)
        total_rewards_right.append(episode_rewards_right)

    avg_reward_left = np.mean(total_rewards_left)
    avg_reward_right = np.mean(total_rewards_right)

    return avg_reward_left, avg_reward_right

 """

In [ ]:
model_left = PPO.load("/content/drive/My Drive/ppo_tennis_left_test", env=vec_env)
model_right = PPO.load("/content/drive/My Drive/ppo_tennis_left_test", env=vec_env)

Wrapping the env in a VecTransposeImage.
Wrapping the env in a VecTransposeImage.


In [13]:
# Load previous evaluation results if they exist
#iterations, avg_rewards_left, avg_rewards_right = load_evaluation_results(evaluation_results_path)

# Train the agents using self-play
total_timesteps = 2000

for i in range(1):  # 30 iterations of self-play training
    print(f"Self-play iteration {i+1}")

    # Train the left agent
    model_left.learn(total_timesteps=total_timesteps)
    model_left.save(left_model_path)  # Save the left agent

    # Train the right agent
    model_right.learn(total_timesteps=total_timesteps)
    model_right.save(right_model_path)  # Save the right agent


    # Reinitialize the environment for evaluation (to avoid wrapping issues)
    #eval_env = DummyVecEnv([lambda: tennis_v3.env()])
    #eval_env = Monitor(eval_env)


      # Evaluate both agents
    #avg_reward_left, avg_reward_right = evaluate_self_play(model_left, model_right, eval_env, n_eval_episodes=10)

    #iterations.append(i + 1)
    #avg_rewards_left.append(avg_reward_left)
    #avg_rewards_right.append(avg_reward_right)
    #print(f"Evaluation after iteration {i+1}: Left Agent Avg Reward: {avg_reward_left}, Right Agent Avg Reward: {avg_reward_right}")
    #save_evaluation_results(evaluation_results_path, i + 1, avg_reward_left, avg_reward_right)



Self-play iteration 1
-----------------------------
| time/              |      |
|    fps             | 563  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 4096 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 458  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 4096 |
-----------------------------


In [ ]:
"""
# Load previous evaluation results if they exist
iterations, avg_rewards_left, avg_rewards_right = load_evaluation_results(evaluation_results_path)
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(iterations, avg_rewards_left, label='Left Agent')
plt.plot(iterations, avg_rewards_right, label='Right Agent')
plt.xlabel('Iterations')
plt.ylabel('Average Reward')
plt.title('Training Progress')
plt.legend()
plt.show()

"""

In [14]:
# Load and test the models
model_left = PPO.load("/content/drive/My Drive/ppo_tennis_left_test", env=vec_env)
model_right = PPO.load("/content/drive/My Drive/ppo_tennis_left_test", env=vec_env)

video_folder = "logs/videos/"
os.makedirs(video_folder, exist_ok=True)
video_length = 1000
env_id = "tennis_v3"

# Wrap the environment with VecVideoRecorder
recording_env = VecVideoRecorder(
    vec_env, video_folder,
    record_video_trigger=lambda x: x == 0, video_length=video_length,
    name_prefix=f"trained-agent-{env_id}"
)

Wrapping the env in a VecTransposeImage.
Wrapping the env in a VecTransposeImage.


In [15]:
# Reset the environment
obs = recording_env.reset()

# Run the environment for the specified number of steps using the trained model
for _ in range(video_length + 1):
    # Extract observations for each agent
    obs_left = obs[0]
    obs_right = obs[1]

    # Get the actions for each agent
    action_left, _ = model_left.predict(obs_left)
    action_right, _ = model_right.predict(obs_right)

    # Create the actions list
    actions = [action_left, action_right]

    # Step the environment with the combined actions
    obs, _, _, _ = recording_env.step(actions)

# Close the VecVideoRecorder and save the video
recording_env.close()

Saving video to /content/logs/videos/trained-agent-tennis_v3-step-0-to-step-1000.mp4
Moviepy - Building video /content/logs/videos/trained-agent-tennis_v3-step-0-to-step-1000.mp4.
Moviepy - Writing video /content/logs/videos/trained-agent-tennis_v3-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /content/logs/videos/trained-agent-tennis_v3-step-0-to-step-1000.mp4


In [16]:
# Function to display the video
def show_video(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video width="480" height="320" controls><source src="{data_url}" type="video/mp4"></video>')

# Display the video
video_path = os.path.join(video_folder, f"trained-agent-{env_id}-step-0-to-step-{video_length}.mp4")
show_video(video_path)